## Add other cell cluster resolutions

In [87]:
import pandas as pd
import numpy as np

cell_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/singleCellTable.csv')
cell_table = cell_table[['fov', 'segmentation_label', 'cell_meta_cluster', 'centroid0', 'centroid1']]
cell_table

,fov,segmentation_label,cell_meta_cluster,centroid0,centroid1
0,TA501_R3C3,1,Undetermined,22.061069,910.946565
1,TA501_R3C3,2,Undetermined,23.196429,900.053571
2,TA501_R3C3,3,Undetermined,25.727273,922.636364
3,TA501_R3C3,4,Undetermined,37.403226,868.887097
4,TA501_R3C3,5,Epcam_Ecad_KRT18,37.452830,940.271698
...,...,...,...,...,...
1169371,TA587_R9C5,705,Stromal,1019.178571,614.392857
1169372,TA587_R9C5,706,Undetermined,1019.850000,216.800000
1169373,TA587_R9C5,707,Stromal,1019.485981,604.280374
1169374,TA587_R9C5,708,Stromal,1020.934783,500.239130


In [88]:
assignment_dict = {'Endothelial': ['Endothelial'],
                   'Epithelial': ['Epcam_Ecad_KRT18', 'Epcam_Ecad_KRT18_81', 'Epcam_Ecad_KRT7_15_18_81', 'Epcam_Ecad_KRT7_18', 
                                  'Epcam_Ecad_KRT7_18_81', 'Epithelial', 'Epithelial_Epcam'],
                   'Immune': ['APC', 'Immune_other', 'Mast_cells', 'Mono_Mac_CD11c', 'Mono_Mac_CD14', 'Mono_Mac_CD163_206', 'Mono_Mac_CD68', 'Mono_Mac_DC_CD209',
                              'Neutrophils', 'Tcell_CD4', 'Tcell_CD8', 'Tcell_other', 'Tregs'],
                   'Myoep': ['Myoep'],
                   'Stromal': ['Stromal'],
                   'Undetermined': ['Undetermined']}

for new_name in assignment_dict:
    pops = assignment_dict[new_name]
    idx = np.isin(cell_table['cell_meta_cluster'].values, pops)
    cell_table.loc[idx,  'cell_cluster_broad'] = new_name

assignment_dict_2 = {'Endothelial': ['Endothelial'],
                     'Epithelial': ['Epcam_Ecad_KRT18', 'Epcam_Ecad_KRT18_81', 'Epcam_Ecad_KRT7_15_18_81', 'Epcam_Ecad_KRT7_18', 
                                    'Epcam_Ecad_KRT7_18_81', 'Epithelial', 'Epithelial_Epcam'],
                     'Immune_other': ['Immune_other'],
                     'Mast_cells': ['Mast_cells'],
                     'APC': ['APC'],
                     'Mono_Mac_CD11c': ['Mono_Mac_CD11c'],
                     'Mono_Mac_CD14': ['Mono_Mac_CD14'],
                     'Mono_Mac_CD163_206': ['Mono_Mac_CD163_206'],
                     'Mono_Mac_CD68': ['Mono_Mac_CD68'],
                     'Mono_Mac_DC_CD209': ['Mono_Mac_DC_CD209'],
                     'Neutrophils': ['Neutrophils'],
                     'Tcell_CD4': ['Tcell_CD4'],
                     'Tcell_CD8': ['Tcell_CD8'],
                     'Tcell_other': ['Tcell_other'],
                     'Tregs': ['Tregs'],
                     'Myoep': ['Myoep'],
                     'Stromal': ['Stromal'],
                     'Undetermined': ['Undetermined']}

for new_name in assignment_dict_2:
    pops = assignment_dict_2[new_name]
    idx = np.isin(cell_table['cell_meta_cluster'].values, pops)
    cell_table.loc[idx,  'only_immune_metaClusters'] = new_name


# save updated cell table
cell_table=cell_table.rename(columns={'segmentation_label': 'label'})

cell_table.to_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cell_table_clusters.csv', index=False)

In [89]:
cell_table

,fov,label,cell_meta_cluster,centroid0,centroid1,cell_cluster_broad,only_immune_metaClusters
0,TA501_R3C3,1,Undetermined,22.061069,910.946565,Undetermined,Undetermined
1,TA501_R3C3,2,Undetermined,23.196429,900.053571,Undetermined,Undetermined
2,TA501_R3C3,3,Undetermined,25.727273,922.636364,Undetermined,Undetermined
3,TA501_R3C3,4,Undetermined,37.403226,868.887097,Undetermined,Undetermined
4,TA501_R3C3,5,Epcam_Ecad_KRT18,37.452830,940.271698,Epithelial,Epithelial
...,...,...,...,...,...,...,...
1169371,TA587_R9C5,705,Stromal,1019.178571,614.392857,Stromal,Stromal
1169372,TA587_R9C5,706,Undetermined,1019.850000,216.800000,Undetermined,Undetermined
1169373,TA587_R9C5,707,Stromal,1019.485981,604.280374,Stromal,Stromal
1169374,TA587_R9C5,708,Stromal,1020.934783,500.239130,Stromal,Stromal


## Calculation cell distances to each duct in the image

In [90]:
import scipy.ndimage as ndi
from skimage.measure import regionprops_table
from scipy.spatial.distance import cdist
import math

STREL_4: np.ndarray = np.array([
    [0,1,0],
    [1,1,1],
    [0,1,0]
], dtype=np.uint8)


def get_outer_boundary(mask: np.ndarray) -> np.ndarray:
    """ Determines the boundary for a binary region mask by comparing it with its erosion

    Args:
        mask (numpy.ndarray):
            Binary image mask.  This can be many connected regions.
    
    Returns:
        numpy.ndarray:
            - Binary image mask depicting the extracted boundary
    """

    image = mask.astype(np.uint8)
    eroded_image = ndi.binary_erosion(image, STREL_4, border_value=10)
    outer_boundary = image - eroded_image

    return outer_boundary


def closest_node(node, nodes):
    return nodes[cdist([node], nodes).argmin()]

def closest_node2(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)


In [ ]:
from skimage.measure import label
from alpineer import io_utils
from skimage.filters import gaussian
from skimage.morphology import binary_erosion, disk, dilation, binary_dilation

from natsort import natsorted
import pandas as pd
from tqdm.notebook import tqdm
import os
import natsort as ns
import skimage.io as io
from copy import deepcopy

import warnings
warnings.filterwarnings("ignore")

mask_dir = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep'
cell_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cell_table_clusters.csv')
fov_list = io_utils.list_folders(mask_dir, substrs='TA')
fov_list = ns.natsorted(fov_list)

lots_of_ducts, duct_num = [], []
edges_dists, center_dists, ratio_dists = [], [], []
dfs = []
tables = []
mappings = []

with tqdm(total=len(fov_list), desc="Edge Distance Calculation", unit="FOV") \
as progress:
    for fov_num, fov in enumerate(fov_list):
        if fov_num%25==0:
            print(i)

        # read in mask
        mask_path = os.path.join(mask_dir, fov, 'final_masks', 'ducts_labeled.tiff')
        try:
            print(fov)
            ducts_labeled = io.imread(mask_path)
        except:
            print(f"Failed: {fov}")
            continue

        ## GENERATE MYOEP MASKS ##
        # identify separate objects and filter by size
        epi_mask = ducts_labeled.copy()
        epi_mask[epi_mask>0]=1
        mask_sep = label(epi_mask)

        for obj in np.unique(mask_sep[mask_sep>0]):
            area = np.where(mask_sep[mask_sep==obj])
            area = len(np.transpose(area))
            if area < 1500:
                mask_sep[mask_sep==obj] = 0

        object_num = len(np.unique(mask_sep))-1

        if object_num <= 20:
            # gaussian blur of 2
            blurred = gaussian(mask_sep, sigma=2)
            blurred[blurred>0]=1
            blurred = binary_dilation(blurred, disk(5))
        else:
            lots_of_ducts.append(fov)
            duct_num.append(object_num)
            # fill holes, no blur
            blurred = ndi.binary_fill_holes(mask_sep)
            blurred[blurred>0]=1
            blurred = binary_dilation(blurred, disk(5))

        '''
        blurred = gaussian(epi_mask, sigma=2)
        blurred[blurred>0]=1
        blurred = binary_dilation(blurred, disk(5))
        blurred = ndi.binary_fill_holes(blurred)
        blurred=blurred*1
        '''

        mask_sep2 = label(blurred)
        for obj in np.unique(mask_sep2[mask_sep2>0]):
            area = np.where(mask_sep2[mask_sep2==obj])
            area = len(np.transpose(area))
            mask_sep2[mask_sep2==obj] = area

        epi_large_area = 100000

        # erosion of 45 (+20) pixels for larger epithelial masks
        large_epi = np.zeros_like(mask_sep2)
        large_epi[mask_sep2 >= epi_large_area] = 1
        large_ducts = binary_erosion(large_epi, disk(45))
        #large_ducts = binary_erosion(large_epi, disk(65))

        # erosion of 25 pixels (+20)
        small_epi = np.zeros_like(mask_sep2)
        small_epi[np.logical_and(mask_sep2 > 0, mask_sep2 < epi_large_area)] = 1
        small_ducts = binary_erosion(small_epi, disk(25))
        #small_ducts = binary_erosion(small_epi, disk(45))

        # combine into one mask
        duct = large_ducts + small_ducts

        # subtract duct from blurred mask to get myoepithelial mask
        blurred = blurred.astype('int')
        duct = duct.astype('int')
        myoep = np.subtract(blurred, duct)
        myoep[myoep==1] = 255

        # subtract and label by duct num
        if os.path.exists(os.path.join(mask_dir, fov, 'subtract_mask.tiff')):
            subtract_mask = io.imread(os.path.join(mask_dir, fov, 'subtract_mask.tiff'))
            exp = binary_dilation(subtract_mask, disk(10)).astype(subtract_mask.dtype)
            exp = exp*[10]
            exp[exp==0]=1
            exp[exp==10]=0

            good_myoep = exp*(myoep/255)
        else: 
            good_myoep = myoep
            good_myoep = good_myoep/255

        # ducts_labeled = io.imread(os.path.join(mask_dir, fov, "ducts_labeled.tiff"))
        exp = dilation(ducts_labeled, disk(20))
        myoep_labeled = (good_myoep*exp).astype(int)

        io.imsave(os.path.join(mask_dir, fov, 'final_masks', 'myoep_labeled.tiff'), myoep_labeled.astype('uint8'), check_contrast=False)


        ## GENERATE STROMA MASKS ##
        if os.path.exists(os.path.join(mask_dir, fov, 'final_masks', 'ducts_labeled.tiff')):
            gold_mask = io.imread(os.path.join(mask_dir, fov, 'final_masks', 'gold_mask.tiff'))
            gold_mask = gold_mask/255
            
            epi_mask = ducts_labeled.copy()
            epi_mask[epi_mask>0] = 1

            stroma_mask = np.logical_not(epi_mask).astype(np.uint8)
            stroma_mask = stroma_mask - gold_mask
            stroma_mask[stroma_mask<0] = 0

            io.imsave(os.path.join(mask_dir, fov, 'final_masks', 'stroma_mask.tiff'), stroma_mask*255,
                      check_contrast=False)


        ## CELL TO DUCT CENTER/EDGE DIST ##
        fov_table = cell_table[cell_table.fov==fov]

        duct_copy = deepcopy(ducts_labeled)
        duct_copy[duct_copy>0]=1
        outer_boundaries = get_outer_boundary(duct_copy)
        outer_boundaries_labeled = outer_boundaries * ducts_labeled

        # save duct mapping
        ducts = np.unique(ducts_labeled[ducts_labeled>0])
        if len(ducts)==0:
            ducts = [np.nan]
        mapping = pd.DataFrame(
            {
                'fov': [fov]*len(ducts), 
                'DuctNumber': ducts, 
                'idx': range(len(ducts))
        }) 
        mappings.append(mapping)
        
        # get duct centroids and outer boundary
        for i,duct_num in enumerate(ducts):

            object_mask = deepcopy(ducts_labeled)
            object_mask[object_mask != duct_num] = 0
            props = regionprops_table(object_mask, properties=('centroid', 'orientation'))
            duct_centroid = (float(props['centroid-0']), float(props['centroid-1']))
            coords = np.transpose(np.where(outer_boundaries_labeled == duct_num))

            # check each cell distance 
            for cell in fov_table.label:
                # get cell location
                centroid_0 = float(fov_table[fov_table.label==cell]['centroid0'])
                centroid_1 = float(fov_table[fov_table.label==cell]['centroid1'])
                cell_centroid = (centroid_0, centroid_1)
                
                # get distances
                closest_dist = cdist([cell_centroid], coords).min()
                dist_to_center = math.dist(cell_centroid, duct_centroid)

                fov_table.loc[(fov_table.fov==fov) & (fov_table.label==cell), f'distance_to_edge_{str(i)}'] = closest_dist
                fov_table.loc[(fov_table.fov==fov) & (fov_table.label==cell), f'distance_to_center_{str(i)}'] = dist_to_center
                
        tables.append(fov_table)
        progress.update(1)
        
dist_table = pd.concat(tables)
mappings = pd.concat(mappings)

mappings.to_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cami_intermediate_files/duct_mapping.csv', index=False)
dist_table.to_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cami_intermediate_files/cells_dist_to_all_duct_edges.csv', index=False)

In [85]:
## CLOSEST DUCT EDGE STATS ##

import pandas as pd
import numpy as np

mappings = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cami_intermediate_files/duct_mapping.csv')
dist_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cami_intermediate_files/cells_dist_to_all_duct_edges.csv')

edge_dist_cols = [f'distance_to_edge_{i}' for i in range(18)]
dist_table['min_dist'] = dist_table[edge_dist_cols].min(axis=1)
dist_table['min_col'] = dist_table[edge_dist_cols].idxmin(axis='columns')
dist_table['idx'] = [int(str(min_col).split('_')[-1]) if type(min_col) == str else float("nan") for min_col in dist_table['min_col']]
mappings = mappings.rename(columns={'DuctNumber': 'min_duct_num'})
dist_table = dist_table.merge(mappings, on=['fov', 'idx'], how='left')
dist_table['distance_to_duct_edge'] = [dist_table[f'distance_to_edge_{str(int(idx))}'][row_num] if str(idx) != 'nan' else float("nan") for row_num, idx in enumerate(dist_table['idx'])]
dist_table['distance_to_duct_center'] = [dist_table[f'distance_to_center_{str(int(idx))}'][row_num] if str(idx) != 'nan' else float("nan") for row_num, idx in enumerate(dist_table['idx'])]
dist_table['distance_ratio'] = dist_table['distance_to_duct_edge'] / dist_table['distance_to_duct_center']

final_dist_table = dist_table[['fov', 'label', 'cell_meta_cluster', 'cell_cluster_broad', 'only_immune_metaClusters', 'min_duct_num', 'distance_to_duct_edge', 'distance_to_duct_center', 'distance_ratio']]
final_dist_table.to_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cami_intermediate_files/cells_dist_to_duct_edge.csv', index=False)
final_dist_table

/var/folders/xj/4skzzr9101569wyr6sxjvsd00000gq/T/ipykernel_16147/3674606078.py:11: FutureWarning: The behavior of DataFrame.idxmin with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  dist_table['min_col'] = dist_table[edge_dist_cols].idxmin(axis='columns')


,fov,label,cell_meta_cluster,cell_cluster_broad,only_immune_metaClusters,min_duct_num,distance_to_duct_edge,distance_to_duct_center,distance_ratio
0,TA501_R3C3,1,Undetermined,Undetermined,Undetermined,1.0,52.466191,599.578751,0.087505
1,TA501_R3C3,2,Undetermined,Undetermined,Undetermined,1.0,62.987006,591.488081,0.106489
2,TA501_R3C3,3,Undetermined,Undetermined,Undetermined,1.0,42.732011,604.784173,0.070657
3,TA501_R3C3,4,Undetermined,Undetermined,Undetermined,1.0,97.188117,560.416432,0.173421
4,TA501_R3C3,5,Epcam_Ecad_KRT18,Epithelial,Epithelial,1.0,27.496202,608.624724,0.045178
...,...,...,...,...,...,...,...,...,...
1016742,TA587_R12C5,661,Stromal,Stromal,Stromal,2.0,604.838168,829.941169,0.728772
1016743,TA587_R12C5,662,Undetermined,Undetermined,Undetermined,2.0,696.551553,902.333612,0.771945
1016744,TA587_R12C5,663,Stromal,Stromal,Stromal,2.0,602.281403,829.640929,0.725954
1016745,TA587_R12C5,664,Myoep,Myoep,Myoep,2.0,593.778462,824.355057,0.720295


## Assign cells to it's duct number or stroma 

In [ ]:
## ASSIGN DuctNumber ##
import os
import numpy as np
import pandas as pd
import skimage.io as io
from copy import deepcopy
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

mask_dir = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep'
seg_dir = '/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/segmentation/deepcell_output'
cell_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cell_table_clusters.csv')

tables = []
with tqdm(total=len(np.unique(cell_table.fov)), desc="DuctNumber assignment", unit="FOV") \
as progress:
    for fov_num, fov in enumerate(np.unique(cell_table.fov)):
        # read in mask
        mask_path = os.path.join(mask_dir, fov, 'final_masks', 'ducts_labeled.tiff')
        try:
            ducts_labeled = io.imread(mask_path)
        except:
            progress.update(1)
            continue
        
        fov_table = cell_table[cell_table.fov==fov]
        for cell in fov_table.label:
            centroid_0 = int(round(fov_table[fov_table.label==cell]['centroid0'].values[0], 0))
            centroid_1 = int(round(fov_table[fov_table.label==cell]['centroid1'].values[0], 0))
            
            ductnum = ducts_labeled[centroid_0, centroid_1]
            fov_table.loc[(fov_table.fov==fov) & (fov_table.label==cell), f'DuctNumber'] = ductnum
            
        tables.append(fov_table)
        progress.update(1)
        
ductnum_table = pd.concat(tables)
ductnum_table.to_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cami_intermediate_files/cell_DuctNumber.csv', index=False)

## Aggregate cell level stats, image-wide and per duct

In [ ]:
## CELL x CELL average distance aggregation ##

import pandas as pd
import numpy as np
import os

ductnum_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cell_table_DuctNumber.csv')
dist_table = pd.read_csv('/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/spatial_analysis/cell_neighbor_analysis/only_immune_metaClusters_avg_dists-nearest_5.csv')

ductnum_dist_table = ductnum_table.merge(dist_table, on=['fov', 'label'])
ductnum_dist_table.to_csv('/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/spatial_analysis/cell_neighbor_analysis/DuctNumber-only_immune_metaClusters_avg_dists-nearest_5.csv', index=False)

output_dir = '/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/spatial_analysis/cell_neighbor_analysis/fov_wide_threshed_dist_avgs'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

cell_type_col = 'only_immune_metaClusters'

# only consider distances <= 1000
vals = ductnum_dist_table.iloc[:, 4:]
df = vals.mask(vals > 1000)
threshed_df = pd.concat([ductnum_dist_table.iloc[:, :4], df], axis = 1)

# aggregate across image, and by duct
fov_agg = threshed_df.groupby(by=["fov", cell_type_col]).mean().reset_index()
duct_agg = threshed_df.groupby(by=["fov", "DuctNumber", cell_type_col]).mean().reset_index()

fov_agg.to_csv(os.path.join(output_dir, f"{cell_type_col}_fov_averages_5.csv"), index=False)
duct_agg.to_csv(os.path.join(output_dir, f"{cell_type_col}_duct_averages_5.csv"), index=False)

## Generate myoepithelial masks

In [37]:
redo_file = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/duct_removal_updated.csv')

In [60]:
for fov in np.unique(redo_file.fov):
    if fov in ['TA507_R12C1', 'TA585_R8C6', 'TA586_R10C5', 'TA535_R10C7', 'TA538_R11C1', 'TA539_R5C4', 'TA587_R10C4', 
            'TA538_R9C6', 'TA507_R2C5', 'TA586_R1C1', 'TA584_R6C4', 'TA584_R12C4', 'TA536_R11C1', 'TA535_R12C1', 
            'TA538_R11C2', 'TA583_R8C5', 'TA501_R4C7', 'TA503_R1C1', 'TA583_R6C3', 'TA539_R1C1', 'TA537_R6C2', 
            'TA537_R6C5', 'TA535_R11C6', 'TA536_R10C4', 'TA538_R1C1', 'TA538_R3C5']:
        print(fov)

TA537_R6C2
TA537_R6C5
TA538_R9C6
TA583_R8C5
TA586_R10C5


In [ ]:
## Create myoep masks ##
from skimage.measure import label
from alpineer import io_utils
from skimage.filters import gaussian
from skimage.morphology import binary_erosion, disk, dilation, binary_dilation
from natsort import natsorted
from alpineer import io_utils
import os
import warnings
import numpy as np
import pandas as pd 
import skimage.io as io
from copy import deepcopy
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from skimage.measure import label
from alpineer import io_utils
from natsort import natsorted
import shutil


mask_dir = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep'

fov_list = io_utils.list_folders(mask_dir, substrs='TA')

lots_of_ducts, duct_num = [], []
for i, fov in enumerate(fov_list):
    if i%25==0:
        print(i)
        
    # read in mask
    #mask_path = os.path.join(mask_dir, f'{fov}/epi_mask.tiff')
    mask_path = os.path.join(mask_dir, f'{fov}/final_masks/ducts_labeled.tiff')

    try:
        epi_mask = io.imread(mask_path)
        print(fov)
    except:
        print(f"Failed: {fov}")
        continue
        
    epi_mask[epi_mask>0]=1

    '''
    # identify separate objects and filter by size
    mask_sep = label(epi_mask)
    
    for obj in np.unique(mask_sep[mask_sep>0]):
        area = np.where(mask_sep[mask_sep==obj])
        area = len(np.transpose(area))
        if area < 1500:
            mask_sep[mask_sep==obj] = 0
    
    object_num = len(np.unique(mask_sep))-1
    #print(object_num)
    #mask_sep[mask_sep>0]=1
    #plt.imshow(mask_sep)
    
    if object_num <= 20:
        # gaussian blur of 2
        blurred = gaussian(mask_sep, sigma=2)
        blurred[blurred>0]=1
        blurred = binary_dilation(blurred, disk(5))
    else:
        #print(fov+': '+str(object_num))
        lots_of_ducts.append(fov)
        duct_num.append(object_num)
        # fill holes, no blur
        blurred = ndi.binary_fill_holes(mask_sep)
        blurred[blurred>0]=1
        blurred = binary_dilation(blurred, disk(5))'''
    
    blurred = gaussian(epi_mask, sigma=2)
    blurred[blurred>0]=1
    blurred = binary_dilation(blurred, disk(5))
    blurred = ndi.binary_fill_holes(blurred)
    blurred=blurred*1

    mask_sep2 = label(blurred)
    for obj in np.unique(mask_sep2[mask_sep2>0]):
        area = np.where(mask_sep2[mask_sep2==obj])
        area = len(np.transpose(area))
        mask_sep2[mask_sep2==obj] = area
        
    epi_large_area = 100000
    # erosion of 45 (+20) pixels for larger epithelial masks
    large_epi = np.zeros_like(mask_sep2)
    large_epi[mask_sep2 >= epi_large_area] = 1
    large_ducts = binary_erosion(large_epi, disk(45))
    # large_ducts = binary_erosion(large_epi, disk(65))
    
    # erosion of 25 pixels (+20)
    small_epi = np.zeros_like(mask_sep2)
    small_epi[np.logical_and(mask_sep2 > 0, mask_sep2 < epi_large_area)] = 1
    small_ducts = binary_erosion(small_epi, disk(25))
    # small_ducts = binary_erosion(small_epi, disk(45))
    
    # combine into one mask
    duct = large_ducts + small_ducts
    
    # subtract duct from blurred mask to get myoepithelial mask
    blurred = blurred.astype('int')
    duct = duct.astype('int')
    myoep = np.subtract(blurred, duct)
    myoep[myoep==1] = 255
    
    # subtract and label by duct num
    if os.path.exists(os.path.join(mask_dir, fov, 'final_masks', 'subtract_mask.tiff')):
        subtract_mask = io.imread(os.path.join(mask_dir, fov, 'final_masks', 'subtract_mask.tiff'))
        exp = binary_dilation(subtract_mask, disk(10)).astype(subtract_mask.dtype)
        exp = exp*[10]
        exp[exp==0]=1
        exp[exp==10]=0
        
        good_myoep = exp*(myoep/255)
    else: 
        good_myoep = myoep
        good_myoep = good_myoep/255
    
    ducts_labeled = io.imread(os.path.join(mask_dir, fov, 'final_masks', "ducts_labeled.tiff"))
    exp = dilation(ducts_labeled, disk(20))
    myoep_labeled = (good_myoep*exp).astype(int)
    
    io.imsave(os.path.join(mask_dir, f'{fov}/final_masks/myoep_labeled.tiff'), myoep_labeled.astype('uint8'), check_contrast=False)

## Create myoep gridding

In [ ]:
pip install pyamg

In [ ]:
pip install Shapely

In [64]:
## NEW
import os

import skimage.io as io
from skimage.filters import sobel_h, sobel_v, meijering
from skimage.morphology import remove_small_objects, dilation, erosion, medial_axis, convex_hull_image, thin, binary_erosion
from skimage.transform import downscale_local_mean, rescale
from skimage.draw import circle_perimeter

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib.path import Path

from scipy import ndimage as ndi
from scipy.spatial.distance import cdist
from skimage.measure import label
import warnings
import shutil


#from ark.utils import io_utils

%matplotlib inline

from typing import Tuple, Dict
import numpy as np
import scipy.ndimage as ndi
from skimage.morphology import skeletonize

# border connectivity mask
STREL_4: np.ndarray = np.array([
    [0,1,0],
    [1,1,1],
    [0,1,0]
], dtype=np.uint8)


def get_spine(mask: np.ndarray,  trim_thresh=2, trim_plus: bool = False) -> np.ndarray:
    """ Reduces medial axis skeletonization of mask into a one-pixel-wide spine.

    This extracts a one-pixel-wide spine from a binary image mask depicting one connected region.
    Starting from the object's medial axis skeletonization, regions matching the finite set of
    junction points (add link) are tagged.  Additionally all end points are identified from the
    finite set of end points (add link).  The end points are repeatedly trimmed until there are
    no junction points in the skeleton mask, indicating the reduction to a spine.

    Useful for forcing human annotated boundaries to be 1-pixel wide.

    Args:
        mask (numpy.ndarray):
            Binary image mask.  This should only consist of one connected region.
        trim_plus (bool):
            Remove '+' junction artifacts.  Sometimes necessary.  Defaults to False.

    Returns:
        numpy.ndarray:
            - Binary image mask depicting the extracted spine
    """

    # TODO: make sure that mask only contains one connected mask

    # start with medial axis skeleton
    skeleton: np.ndarray = skeletonize(mask)
    # skeleton: np.ndarray = medial_axis(mask)

    return skeleton

def get_outer_boundary(mask: np.ndarray) -> np.ndarray:
    """ Determines the boundary for a binary region mask by comparing it with its erosion

    Args:
        mask (numpy.ndarray):
            Binary image mask.  This can be many connected regions.
    
    Returns:
        numpy.ndarray:
            - Binary image mask depicting the extracted boundary
    """

    image = mask.astype(np.uint8)
    eroded_image = ndi.binary_erosion(image, STREL_4, border_value=0)
    outer_boundary = image - eroded_image

    return outer_boundary

def get_end_points(mask: np.ndarray, expected: int = None) -> tuple:
    """ Lists the end points of a given one pixel wide binary area mask

    Args:
        mask (np.ndarray):
            Binary image mask.  Should be one pixel wide
        expected (int | None):
            Number of expected end points.  If None, any number of end points can be found
    """

    # candidate end points must be positive and have a convolution of two on a one pixel wide spine
    candidates: np.ndarray = \
        ndi.correlate(mask.astype(np.uint8), np.ones((3,3)), mode='constant') == 2
    candidates *= mask.astype(np.uint8)

    coords_array = np.argwhere(candidates)
    num_found = coords_array.shape[0]
    if expected is not None and num_found != expected:
        num_found = num_found
        raise ValueError(f'Expected to find {expected} end point{"s" if expected == 1 else ""} '
                         + f'but {num_found} {"were" if num_found == 1 else "was"} found'
                        )
    
    return tuple(map(tuple, coords_array))

from typing import List, Union, Tuple
import numpy as np
from scipy import sparse
from pyamg.gallery import poisson
from pyamg import ruge_stuben_solver

def _set_rows_to_identity(mat: np.ndarray, row_numbers: List[int]) -> np.ndarray:
    """ Sets rows of a matrix to match that of the identity

    Useful for setting Dirchlet boundary conditions on linear systems

    Args:
        mat (np.ndarray):
            Matrix of interest.  Preferably sparse
        row_numbers (list[int]):
            List of rows to set to the identity

    Returns:
        np.ndarray:
            Transformed matrix. Result will be sparse if input `mat` is sparse.
    """
    ndofs = mat.shape[0]

    # construct sparse identity, with zero'd rows of interest
    id_mask: np.ndarray = np.ones((ndofs))
    id_mask[row_numbers] = 0
    id_mask = sparse.dia_matrix((id_mask, 0), shape=(ndofs, ndofs))

    # zero rows of interest
    mat = id_mask.dot(mat)

    # fill rows of interest with identity
    new_diag_entries: np.ndarray = np.zeros((ndofs))
    new_diag_entries[row_numbers] = 1
    id_mask = sparse.dia_matrix((new_diag_entries, 0), shape=(ndofs, ndofs))
    mat = mat + id_mask

    return mat

def set_diag(Amat,bc_id):
    ndofs = Amat.shape[0]
    diago = Amat.diagonal()

    uno = np.ones((ndofs))
    uno[bc_id] = 0
    uno = sparse.dia_matrix((uno,0), shape = (ndofs,ndofs))
    # up to here I delete the rows
    # I multiply A by an identity matrix
    # where i set to zero the rows I want
    # to delete
    Amat = uno.dot(Amat)
    new_diag_entries = np.zeros((ndofs))
    # here I set the diagonal entries
    # equals to the value on the diagonal.
    # Use the second line if you want to
    # to set the diagonal entry to one
    new_diag_entries[bc_id] = diago[bc_id]
    #new_diag_entries[bc_id] = uno[bc_id]
    uno = sparse.dia_matrix((new_diag_entries,0), shape = (ndofs,ndofs))
    Amat = Amat + uno # here I set the diagonal entry
    return Amat

def solve_potential(rho: np.ndarray, domain: np.ndarray, tolerance: float=1e-10,
                    return_residual: bool=False) -> Union[np.ndarray, Tuple[np.ndarray, float]]:
    """ Generates and solves a discretized 2D Poisson equation (Ax = `rho`) on a given domain

    Args:
        rho (np.ndarray):
            Left hand side of the system
        domain (np.ndarray):
            Binary image depicting the desired domain
        tolerance (float):
            Error tolerance for the solution
        return_residual (bool):
            Whether or not to return the residual with the solution
    
    Returns:
        np.ndarray or Tuple[np.ndarray, float]:
            solution, or solution tupled with residual

    """
    # generate laplacian matrix
    rho_flat=rho.ravel()
    A = poisson(rho.shape, format='csr')
    
    # notebook testing
    A_beta = _set_rows_to_identity(A, domain)

    # populate entries with identity on non-domain areas
    # row_ids = (np.argwhere(domain == 0).T)[0]
    #A_beta = _set_rows_to_identity(A, row_ids)
    #A_beta = set_diag(A, row_ids)
    #print(np.unique(A-A_beta))
    
    #rho = np.transpose(rho_flat)
    # create multigrid solver
    ml = ruge_stuben_solver(A_beta)

    # generate solution
    x = ml.solve(rho_flat, tol=tolerance)
    x_big = np.reshape(x, rho.shape)

    # compute and return the residual
    if return_residual:
        res = np.linalg.norm(rho - (A_beta*x).reshape(rho.shape))
        return x_big, res

    # return solution
    return x_big

import numpy as np
from skimage.filters.edges import _generic_edge_filter

SCHARR_EDGE_5x5: np.ndarray = np.array([24, 80, 0, -80, -24])
SCHARR_SMOOTH_5x5: np.ndarray = np.array([7, 63, 116, 63, 7]) / 256

def vector_gradient(image: np.ndarray) -> np.ndarray:
    """ Compute image gradient via five point Scharr filter

    [1] Scharr, Hanno, 2000, Optimal Operators in Digital Image Processing 
    """

    grad = np.zeros((*image.shape, 2))

    grad[:, :, 0] = _generic_edge_filter(
        image, smooth_weights=SCHARR_SMOOTH_5x5, edge_weights=SCHARR_EDGE_5x5, axis=0
    )
    grad[:, :, 1] = _generic_edge_filter(
        image, smooth_weights=SCHARR_SMOOTH_5x5, edge_weights=SCHARR_EDGE_5x5, axis=1
    )

    return grad


from typing import Tuple, Union
import numpy as np

#from isharc import mask_utils, potential_solver, gradient_utils

def polar(region_mask: np.ndarray,  trim_thresh=2, inner_boundary: np.ndarray = None,
          ns_intercepts: Tuple[Tuple[int, int], Tuple[int, int]] = None,
          thin_boundary: bool = False, show_residual: bool = False,
          return_angle: bool = True, skip_outer=False) -> Union[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
    """ Assigns polar-esque coordinates to the specified region via harmonic coordinatization

    Args:
        region_mask (np.ndarray):
            Binary mask depicting the region of interest
        inner_boundary (np.ndarray | None):
            Optional binary mask depicting the inner boundary. If None, inner boundary is set to
            the spine of the region, computed via `mask_utils.get_spine`
        ns_intercepts (tuple):
            Optional intercepts for the 'north' (theta = pi/2) and 'south' (theta = 3*pi/2) axes.
            If None, intercepts are set to the edge points of the `inner_boundary`
        thin_boundary (bol)
            If True, provided `inner_boundary` is reduced to the one-pixel wide spine
        return_angle (bool):
            If True, angular maps will be computed.  Otherwise, only the radius map will be
            returned
    Returns:
        np.ndarray | Tuple[np.ndarray, np.ndarray]:
            Tuple of x-y to harmonic radius map and x-y to harmonic angle (theta) map as image
            arrays
    """
    
    # construct inner boundary
    if inner_boundary is None:
        inner_boundary = get_spine(region_mask, trim_thresh)
    elif thin_boundary:
        inner_boundary = get_spine(inner_boundary, trim_thresh)

    # generate outer boundary
    if skip_outer:
        outer_boundary = np.zeros(shape=region_mask.shape)
    else:
        outer_boundary = get_outer_boundary(region_mask)
    
    # construct rho
    # TODO: consider normalizing by perimeter for inner boundary, instead of area
    rho: np.ndarray = \
        (1.0 / np.sum(region_mask)) * (
            (outer_boundary.astype(np.float32) / (np.sum(outer_boundary)+1))
            - (inner_boundary.astype(np.float32) / np.sum(inner_boundary))
        )

    # notebook test
    emr = region_mask.ravel()
    bmb = outer_boundary.ravel()
    cb = inner_boundary.ravel()

    row_ids = np.argwhere(((emr == 1) * (bmb + cb == 0)) == 0).T
    row_ids = row_ids[0]
    
    # remove inner boundary from solve domain
    #domain: np.ndarray = (region_mask - inner_boundary) > 0
    #plt.imshow(domain)
    #plt.imshow(rho)
    
    #plt.imshow(inner_boundary)
    #plt.imshow(outer_boundary)
    
    # solve for potential
    potential, residual = solve_potential(rho, row_ids, return_residual=True)
    #print(potential, residual)
    if show_residual:
        print(residual)

    #plt.imshow(potential)
    # return radius map
    if not return_angle:
        return (potential + 1) / 2, inner_boundary

    gradient = vector_gradient(potential)
    norms: np.ndarray = np.linalg.norm(gradient, axis=2)
    normalized_gradient: np.ndarray = gradient / norms[:, :, np.newaxis]
    
    # extract intercepts
    if ns_intercepts is None:
        ns_intercepts = get_end_points(inner_boundary.astype(np.float32), expected=2)

    north_coords, south_coords = ns_intercepts
    
    return north_coords, south_coords
    
from scipy.spatial.distance import cdist
from shapely.geometry import Point, LineString
import matplotlib.pyplot as plt
from copy import deepcopy

def closest_node(node, nodes):
    return nodes[cdist([node], nodes).argmin()]

def closest_node2(node, nodes):
    nodes = np.asarray(nodes)
    dist_2 = np.sum((nodes - node)**2, axis=1)
    return np.argmin(dist_2)

import matplotlib.colors as colors
def set_minimum_color_for_colormap(cmap, default=(0, 0, 0, 1)):
    """ Changes minimum value in provided colormap to black (#000000) or provided color
    This is useful for instances where zero-valued regions of an image should be
    distinct from positive regions (i.e transparent or non-colormap member color)
    Args:
        cmap (matplotlib.colors.Colormap):
            matplotlib color map
        default (Iterable):
            RGBA color values for minimum color. Default is black, (0, 0, 0, 1).
    Returns:
        matplotlib.colors.Colormap:
            corrected colormap
    """
    cmapN = cmap.N
    corrected = cmap(np.arange(cmapN))
    corrected[0, :] = list(default)
    return colors.ListedColormap(corrected)


In [ ]:
from alpineer import io_utils
from tqdm.notebook import tqdm
import natsort as ns

# Shape analysis mask generation
p = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep/'
fovs = io_utils.list_folders(p, substrs='TA')
fovs = ns.natsorted(fovs)

spine_space = 10

with tqdm(total=len(fovs), desc="Shape Analysis", unit="FOVs") as progress:
    for fov in fovs:
        if os.path.exists(p + f'/Shape_analysis-{spine_space}/{fov}'):
            progress.update(1)
            continue
        try:
            m = io.imread(p+fov+'/final_masks/myoep_labeled.tiff')
        except:
            print(f'***** {fov} failed *****')
            progress.update(1)
            continue
        example_mask = deepcopy(m)
        example_mask[example_mask>0] = 1
        mask_sep = label(example_mask)

        ob_total = np.zeros_like(mask_sep)
        sections_total = np.zeros_like(mask_sep)
        spine_total = np.zeros_like(mask_sep)
        potential_total = np.zeros_like(mask_sep)

        for obj_num in np.unique(mask_sep[mask_sep>0]):
            cont_mask = deepcopy(mask_sep)
            cont_mask[cont_mask!=obj_num] = 0
            cont_mask[cont_mask==obj_num] = 1

            # get spine
            spine = get_spine(cont_mask)

            # equidistant spine points
            coords = np.nonzero(spine == True)
            if len(coords[0]) == 0:
                continue
            bps = []
            bps.append([coords[0][0], coords[1][0]])
            coords_T = np.transpose(coords)

            for p_coords in coords_T:
                if (p_coords == coords_T).all(1).any():
                    if not (np.linalg.norm(p_coords - bps, axis=1) < spine_space).any().all():
                        # take the first one and continue
                        bps.append(list(p_coords))

            points = np.where(cont_mask>0)
            spine_pts = {}
            sections = np.zeros_like(mask_sep)

            for point in np.transpose(points):
                # Find the nearest point on the spine
                spine_pt = closest_node2(point, bps)
                sections[point[0], point[1]] = spine_pt+1

            sections_total = np.where(sections == 0, sections_total, sections)

            # potential gradient
            try:
                potential, spiney = polar(cont_mask, return_angle=False)
            except:
                print(f'***** {fov} failed *****')
                break

            spine_total[spiney==1] = 1
            non_zero = np.isfinite(potential)
            potential[potential == 0] = np.mean(potential[non_zero * (potential != 0)])
            rescale_potential = potential - np.min(potential[np.isfinite(potential)])
            rescale_potential = rescale_potential / np.max(rescale_potential[np.isfinite(potential)])
            equipotential_submasks = np.around(rescale_potential, decimals=1)
            equipotential_submasks = equipotential_submasks*10 * cont_mask

            # binning
            for indx, bin in enumerate([(2,3), (4,5), (6,7), (8,9), (10,10)]):
                equipotential_submasks[np.logical_and(equipotential_submasks>=bin[0], equipotential_submasks<=bin[1])] = indx+2

            # set spine pixels = 1
            equipotential_submasks = np.where(equipotential_submasks == 0, cont_mask, equipotential_submasks)

            potential_total = np.where(potential_total == 0, equipotential_submasks, potential_total)

        warnings.filterwarnings("ignore")
        fov_path = p + f'/Shape_analysis-{spine_space}/{fov}'
        if not os.path.exists(fov_path):
            os.makedirs(fov_path)
        print(fov)

        io.imsave(fov_path+'/myoep_labeled.tiff', (m).astype('int16'))
        io.imsave(fov_path+'/potential.tiff', (potential_total).astype('int16'))
        io.imsave(fov_path+'/sections.tiff', (sections_total).astype('int16'))
        io.imsave(fov_path+'/spines.tiff', (spine_total).astype('int16'))
        
        progress.update(1)


## Caclulate myoep shape stats

In [ ]:
# signal computation
import warnings
warnings.filterwarnings("ignore")
import os
from scipy.spatial import distance
import math
from alpineer import io_utils
from skimage.measure import label
from scipy.ndimage.filters import gaussian_filter
from alpineer import io_utils
from tqdm.notebook import tqdm
import natsort as ns
import statistics


p = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep/'

#shape_dir = p + '/Shape_analysis-20/'
#save_dir = p + 'signal_subset/'
width = 10
fovs = io_utils.list_folders(p + f'Shape_analysis-{width}', substrs='TA')
fovs = ns.natsorted(fovs)

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

#chan = "SMA"
thresh = 0.50
width=10

total_stats = []

with tqdm(total=len(fovs), desc="Shape Analysis", unit="FOVs") as progress:
    for i, fov in enumerate(fovs):
        print(f"{fov}")

        shape_dir = p + f'Shape_analysis-{width}/' + fov
        coverages, thickness, ducts, fov_names = [], [], [], []
        thick_var = []
        cont_bins, cont_sects = [], []
        spine_lengths = []

        if not os.path.exists(shape_dir +'/myoep_labeled.tiff'):
            print(f'failed: {fov}')
            continue
            
        sma_signal_no_blur = io.imread(p+fov+f'/final_masks/20240131_myoep_composite_denoised.tiff')
        sma_signal=gaussian_filter(sma_signal_no_blur, sigma=0.5, radius=1)
        sma_signal[sma_signal>0]=1 

        # read in masks
        cont_mask = io.imread(shape_dir+'/myoep_labeled.tiff')
        section_mask = io.imread(shape_dir+'/sections.tiff')
        potential_mask = io.imread(shape_dir+'/potential.tiff')
        spine_mask = io.imread(shape_dir+'/spines.tiff')

        # change all vals to 1 for an area mask
        area_mask = deepcopy(cont_mask)
        area_mask[area_mask>0] = 1
        #total_area = np.sum(area_mask[area_mask>0]) 

        sma_pos = np.zeros_like(cont_mask)
        wts=[]
        
        fov_sect_pos = np.zeros_like(cont_mask)

        # iterated thru each duct
        for duct_num in np.unique(cont_mask[cont_mask>0]):
            thick_weights = []

            # take only grids for current duct
            obj_mask = deepcopy(cont_mask)
            obj_mask[obj_mask!=duct_num] = 0
            obj_mask[obj_mask==duct_num] = 1

            # calculate duct area
            duct_size = np.sum(obj_mask[obj_mask>0])
            duct_sect_pos = np.zeros_like(cont_mask)

            # go over each section
            overlap = obj_mask * section_mask
            for sect_num in np.unique(overlap[overlap>0]):
                centroids = []
                sma_section = np.zeros_like(cont_mask)

                # zero out non-current sections, set current==1
                sect_mask = deepcopy(overlap)
                sect_mask[sect_mask!=sect_num] = 0
                sect_mask[sect_mask==sect_num] = 1

                # go thru each potential region ID
                overlap2 = obj_mask * sect_mask * potential_mask
                for pot_num in np.unique(overlap2[overlap2>0]):
                    # zero out non-current potential regions, set current==1
                    pot_mask = deepcopy(overlap2)
                    pot_mask[pot_mask!=pot_num] = 0
                    pot_mask[pot_mask==pot_num] = 1 

                    # iterate over seperate sections labeled with same int
                    grid_labels = label(pot_mask)
                    for i in np.unique(grid_labels[grid_labels>0]):
                        grid_arr = deepcopy(grid_labels)
                        grid_arr[grid_arr!=i] = 0
                        grid_arr[grid_arr==i] = 1 

                        # subset image for myoep pixels only in the current grid, take average of signal vals
                        sma_grid = sma_signal[grid_arr>0]
                        avg_sma = sma_grid.mean()

                        if avg_sma > thresh:
                            # if meet thresh, add grid to overall & section specific
                            sma_pos = np.where(grid_arr == 0, sma_pos, grid_arr)
                            sma_section = np.where(grid_arr == 0, sma_section, grid_arr)
                
                # if at least one grid pos, the section is sma pos
                if len(sma_section[sma_section>0])!=0:
                    duct_sect_pos = np.where(sect_mask == 0, duct_sect_pos, sect_mask)

                # FIX THICKNESS
                thickness_sect = np.sum(sma_section)
                thick_weights.append(thickness_sect)
            
            thick_weights = np.array(thick_weights)
            thick_weights = thick_weights[np.nonzero(thick_weights)]
            
            # overall positive section mask
            fov_sect_pos = np.where(duct_sect_pos == 0, fov_sect_pos, duct_sect_pos)

            # sum the area of coverage per duct, take percent of whole duct area
            duct_coverage = np.sum(duct_sect_pos)
            coverage_perc = duct_coverage / np.sum(obj_mask)

            # average the area of thickness per duct,
            avg_thickness = np.mean(thick_weights)

            # add stats to df
            fov_names.append(fov)
            ducts.append(duct_num)
            coverages.append(coverage_perc)
            thickness.append(avg_thickness)
            
            ## additional measures
            # variance per duct
            if len(thick_weights)>0:
                thick_var.append(statistics.variance(thick_weights))
            else:
                thick_var.append(np.nan)
            
            # number of continuous objects (bin level and section level)
            connected_bins = len(np.unique((label(sma_pos*obj_mask))))-1
            connected_sections = len(np.unique((label(duct_sect_pos))))-1
            cont_bins.append(connected_bins)
            cont_sects.append(connected_sections)
            
            # spine length for current duct
            spine_obj = spine_mask * obj_mask
            spine_obj[spine_obj>0] = 1
            spine_lengths.append(np.sum(spine_obj))
        
        
        fov_stats = pd.DataFrame(zip(fov_names, ducts, coverages, thickness, thick_var, cont_bins, cont_sects, spine_lengths), 
                                 columns=["fov", "DuctNumber", "myoep_percent_coverage", "myoep_thickness", "thickness_var",
                                          "continuous_bins", "continous_sections", "spine_length"]) 
        total_stats.append(fov_stats)
        
        # save positive myoep grids
        plt.imsave(shape_dir+f'/grids-{thresh}.png', sma_pos)
        io.imsave(shape_dir+f'/grids_mask-{thresh}.tiff', sma_pos.astype('int16'))
        io.imsave(shape_dir+f'/positive_sections-{thresh}.tiff', fov_sect_pos.astype('int16'))
        progress.update(1)

all_stats = pd.concat(total_stats)
all_stats.to_csv(p + f'/Shape_analysis-{width}' + f'/myoep_stats_width_{width}-thresh_{thresh}.csv', index=False)

In [ ]:
## gaps lengths
import os
from alpineer import io_utils
from skimage.measure import label
from tqdm.notebook import tqdm
import natsort as ns
import skimage.io as io
from copy import deepcopy
import numpy as np
import pandas as pd
import statistics


p = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep/'

width=10
fovs = io_utils.list_folders(p + f'Shape_analysis-{width}', substrs='TA')
fovs = ns.natsorted(fovs)

total_stats = []

with tqdm(total=len(fovs), desc="Shape Analysis", unit="FOVs") as progress:
    for i, fov in enumerate(fovs):
        fov_names, ducts  = [], []

        shape_dir = p + f'Shape_analysis-{width}/' + fov

        if not os.path.exists(shape_dir +'/myoep_labeled.tiff'):
            print(f'failed: {fov}')
            continue
        cont_mask = io.imread(shape_dir+'/myoep_labeled.tiff')
        spine_mask = io.imread(shape_dir+'/spines.tiff')
        cont_sects = io.imread(shape_dir+'/positive_sections-0.5.tiff')
        
        cont_gaps = np.logical_not(cont_sects).astype(int)
        cont_gaps = cont_gaps * cont_mask

        spine_numbered = cont_mask * spine_mask

        duct_cont_mean, duct_cont_med, duct_cont_std, duct_cont_max, duct_cont_min = [], [], [], [], []
        # iterated thru each duct
        for duct_num in np.unique(spine_numbered[spine_numbered>0]):
            cont_lengths = []
            
            obj_mask = deepcopy(spine_numbered)
            obj_mask[obj_mask!=duct_num] = 0
            obj_mask[obj_mask==duct_num] = 1

            cont_duct_sects = cont_gaps * obj_mask
            cont_labels = label(cont_duct_sects)
            for i in np.unique(cont_labels[cont_labels>0]):
                cont_sect = deepcopy(cont_labels)
                cont_sect[cont_sect!=i] = 0
                cont_sect[cont_sect==i] = 1
                cont_lengths.append(np.sum(cont_sect))

            cont_lengths = np.array(cont_lengths, dtype=np.float64)
            fov_names.append(fov)
            ducts.append(duct_num)

            # variance of cont lengths
            if len(cont_lengths)>1:
                duct_cont_mean.append(np.mean(cont_lengths))
                duct_cont_med.append(np.median(cont_lengths))
                duct_cont_std.append(statistics.stdev(cont_lengths))
                duct_cont_max.append(np.max(cont_lengths))
                duct_cont_min.append(np.min(cont_lengths))
            elif len(cont_lengths)==1:
                duct_cont_mean.append(np.mean(cont_lengths))
                duct_cont_med.append(np.median(cont_lengths))
                duct_cont_std.append(np.nan)
                duct_cont_max.append(np.max(cont_lengths))
                duct_cont_min.append(np.min(cont_lengths))
            else:
                duct_cont_mean.append(np.nan)
                duct_cont_med.append(np.nan)
                duct_cont_std.append(np.nan)
                duct_cont_max.append(np.nan)
                duct_cont_min.append(np.nan)

        fov_stats = pd.DataFrame(zip(fov_names, ducts, duct_cont_mean, duct_cont_med, duct_cont_std, duct_cont_max, duct_cont_min), 
                                 columns=["fov", "DuctNumber", "gap_length_mean", "gap_length_median", "gap_length_std", 
                                          "gap_length_max", "gap_length_min"]) 

        total_stats.append(fov_stats)
        progress.update(1)

all_stats = pd.concat(total_stats)
all_stats.to_csv(p + f'/Shape_analysis-{width}' + f'/myoep_gaps_stats.csv', index=False)

## Fiber Stats

In [151]:
fiber_table = pd.read_csv(os.path.join(, "FAP_all/fiber_object_table.csv"))
dist_table = pd.read_csv(os.path.join(fiber_dir, "FAP_all/FAP_fibers_dist_to_duct_edge.csv"))
#mapping = pd.read_csv(os.path.join(fiber_dir, "duct_mapping.csv"))
closest_duct = pd.read_csv(os.path.join(fiber_dir, "FAP_fibers_closest_duct.csv"))
objects_table = pd.read_csv(os.path.join(fiber_dir, "FAP_fiber_object_table.csv"))
angle_table = pd.read_csv(os.path.join(fiber_dir, "FAP_fiber_object_table-duct_alignment-length_ratio.csv"))

#stats_per_duct = pd.read_csv(os.path.join(fiber_dir, "FAP_fiber_stats_per_duct.csv"))

In [91]:
import os
import numpy as np
import pandas as pd
import skimage.io as io
from copy import deepcopy
import matplotlib.pyplot as plt
from skimage.measure import regionprops_table

In [95]:
# channel = 'FAP'
channel = 'COL1A1'

base_dir = '/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS'
fiber_dir = f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}'

mask_dir = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep/'

In [96]:
fiber_table = pd.read_csv(os.path.join(fiber_dir, f"{channel}_all/fiber_object_table.csv"))

In [ ]:
# check each fiber for distance to closest duct edge
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")
tables = []

with tqdm(total=len(np.unique(fiber_table.fov)), desc="Edge Distance Calculation", unit="FOVs") \
as progress:
    for fov in np.unique(fiber_table.fov):
        fov_table = fiber_table[fiber_table.fov==fov]

        try:
            fov_mask = io.imread(os.path.join(mask_dir, fov, 'ducts_labeled.tiff'))
        except:
            print(f"Failed: {fov}")
            progress.update(1)
            continue
        duct_copy = deepcopy(fov_mask)
        duct_copy[duct_copy>0]=1
        outer_boundaries = get_outer_boundary(duct_copy)
        outer_boundaries_labeled = outer_boundaries * fov_mask
        
        ducts = np.unique(fov_mask[fov_mask>0])
        
        # get duct centroids and outer boundary
        for i, duct_num in enumerate(np.sort(ducts)):
            object_mask = deepcopy(fov_mask)
            object_mask[object_mask != duct_num] = 0
            props = regionprops_table(object_mask, properties=('centroid', 'orientation'))
            duct_centroid = (float(props['centroid-0']), float(props['centroid-1']))

            coords = np.transpose(np.where(outer_boundaries_labeled == duct_num))

            # check each cell distance 
            for fiber in fov_table.label:
                # get fiber location
                centroid_0 = float(fov_table[fov_table.label==fiber]['centroid-0'])
                centroid_1 = float(fov_table[fov_table.label==fiber]['centroid-1'])
                fiber_centroid = (centroid_0, centroid_1)
                
                # get distances
                closest_dist = cdist([fiber_centroid], coords).min()
                dist_to_center = math.dist(fiber_centroid, duct_centroid)

                fov_table.loc[(fov_table.fov==fov) & (fov_table.label==fiber), f'distance_to_edge_{str(i)}'] = closest_dist
                fov_table.loc[(fov_table.fov==fov) & (fov_table.label==fiber), f'distance_to_center_{str(i)}'] = dist_to_center
                
        #fiber_table.to_csv('/Users/csowers/Documents/DCIS_dists/' + f'{fov}_dists.csv' , index=False)
        tables.append(fov_table)
        progress.update(1)
        
fiber_dist_table = pd.concat(tables)
fiber_dist_table.to_csv(os.path.join(fiber_dir, f'{channel}_all/{channel}_fibers_dist_to_duct_edge.csv'), index=False)

In [98]:
cell_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cell_table_clusters.csv')
mappings = []

for fov in np.unique(table.fov):
    fov_table = cell_table[cell_table.fov==fov]
    fov_table = fov_table[fov_table.DuctNumber>0]
    ducts = np.array(np.unique(fov_table.DuctNumber))
    ducts = np.sort(ducts)
    mapping = pd.DataFrame(
        {
            'fov': [fov]*len(ducts), 
            'DuctNumber': ducts, 
            'index': range(len(ducts))
        }) 
    
    mappings.append(mapping)
mappings = pd.concat(mappings)
mappings.to_csv('/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/duct_mapping.csv', index=False)

In [100]:
# duct mapping to column
channel = 'FAP'
# channel = 'COL1A1'

fiber_dir = f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}'
fiber_dist_table = pd.read_csv(os.path.join(fiber_dir, f'{channel}_all/{channel}_fibers_dist_to_duct_edge.csv'))

# get min duct dist
cols = [f'distance_to_edge_{indx}' for indx in range(18)]
fiber_dist_table['min_edge_dist'] = fiber_dist_table[cols].min(axis='columns') # minimum value
fiber_dist_table['min_duct_index'] = fiber_dist_table[cols].idxmin(axis = 'columns').str[-1:]

mappings = mappings.rename(columns={'index': 'min_duct_index', 'DuctNumber': 'min_duct_num'})

fiber_dist_table['min_duct_index']=fiber_dist_table['min_duct_index'].astype(float)
mappings['min_duct_index']=mappings['min_duct_index'].astype(float)

fin = fiber_dist_table.merge(mappings, on=['fov', 'min_duct_index'])
min_dists = fin[['fov', 'label', 'min_edge_dist', 'min_duct_index', 'min_duct_num']]
min_dists.to_csv(os.path.join(fiber_dir, f'{channel}_fibers_closest_duct.csv'), index=False)

table = pd.read_csv(os.path.join(fiber_dir, f'{channel}_all/fiber_object_table.csv'))
stats=table.merge(min_dists, on=['fov', 'label'], how='left')
stats.to_csv(os.path.join(fiber_dir, f'{channel}_fiber_object_table.csv'), index=False)

In [ ]:
## Myoep section orientations ##

from tqdm.notebook import tqdm
import warnings
import skimage.io as io
from skimage import morphology
from ark import settings
warnings.filterwarnings("ignore")

cell_table = pd.read_csv('/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/Tables/cell_table_clusters.csv')

edges_dists, center_dists, ratio_dists = [], [], []
dash_df = []

with tqdm(total=len(np.unique(cell_table.fov)), desc="Edge Distance Calculation", unit="FOV") \
as progress:
    for fov in np.unique(cell_table.fov):
        fov_table = cell_table[cell_table.fov==fov]

        try:
            fov_mask = io.imread(os.path.join(mask_dir, fov, 'ducts_labeled.tiff'))
        except:
            print(f"Failed: {fov}")
            progress.update(1)
            continue
            
        fov_mask_blur = morphology.remove_small_holes(fov_mask, 100000)
        fov_mask_blur = gaussian_filter(fov_mask_blur, sigma=4)
            
        duct_copy = deepcopy(fov_mask_blur)
        duct_copy[duct_copy>0]=1
        outer_boundaries = get_outer_boundary(duct_copy)
        outer_boundaries_labeled = outer_boundaries * fov_mask
        
        for duct_num in np.unique(outer_boundaries_labeled[outer_boundaries_labeled>0]):
            outer_copy = deepcopy(outer_boundaries_labeled)
            outer_copy[outer_copy!=duct_num]=0
            
            for i in [100*idx for idx in range(41)]:
                if i < outer_boundaries_labeled.shape[0]:
                    outer_copy[i:i+10,] = 0
                    outer_copy[:,i+20:i+30] = 0

            outer_copy_labeled = label(outer_copy)
            df = pd.DataFrame(regionprops_table(outer_copy_labeled, properties=settings.FIBER_OBJECT_PROPS))
            df['DuctNumber'] = duct_num
            df['fov'] = fov
            
            dash_df.append(df)
        progress.update(1)
            
dash_df = pd.concat(dash_df)
dash_df_threshed = dash_df[dash_df.area>5]
dash_df_threshed.to_csv('/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/duct_sectioned_angles.csv', index=False)

In [ ]:
## Measure fiber alignment to closest myoep section ##

from scipy.spatial.distance import cdist
import xarray as xr

# channel = 'FAP'
channel = 'COL1A1'

fiber_table_all = pd.read_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table.csv')
dash_df = pd.read_csv('/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/duct_sectioned_angles.csv')

fiber_table_og = fiber_table_all[~fiber_table_all.min_duct_num.isna()]
'''fiber_table_og = fiber_table_og[fiber_table_og.fov!='TA502_R1C5']
fiber_table_og = fiber_table_og[fiber_table_og.fov!='TA504_R7C3']
fiber_table_og = fiber_table_og[fiber_table_og.fov!='TA583_R9C5']
fiber_table_og = fiber_table_og[fiber_table_og.fov!='TA584_R8C5']'''
filtered_lengths = fiber_table_og[(fiber_table_og['major_axis_length'].values / fiber_table_og['minor_axis_length'].values)
                                  >= 2]
fiber_table = filtered_lengths.reset_index()

with tqdm(total=len(np.unique(fiber_table.fov)), desc="Edge Distance Calculation", unit="FOV") \
as progress:
    for fov in np.unique(fiber_table.fov):
        fov_table = fiber_table[fiber_table.fov==fov]
        fov_segments = dash_df[dash_df.fov==fov]
        
        for duct_num in np.unique(fov_table.min_duct_num):
            duct_table = fov_table[fov_table.min_duct_num==duct_num]
            duct_segments = fov_segments[fov_segments.DuctNumber==duct_num]
        
            '''centroid_0 = float(duct_table[duct_table.label==fiber]['centroid0'])
            centroid_1 = float(duct_table[duct_table.label==fiber]['centroid1'])
            fiber_centroid = (centroid_0, centroid_1)'''
            fiber_centroids = [(row['centroid-0'], row['centroid-1']) for indx, row in duct_table.iterrows()]
            fiber_labels = list(duct_table.label)
                
            dash_centroids = [(row['centroid-0'], row['centroid-1']) for indx, row in duct_segments.iterrows()]
            dash_labels = list(duct_segments.label)

            # get distances
            #closest_dist = cdist(fiber_centroids, dash_centroids).astype(np.float32)
            dist_matrix = cdist(fiber_centroids, dash_centroids).astype(np.float32)
            dist_mat_xarr = xr.DataArray(dist_matrix, coords=[fiber_labels, dash_labels])
            #print(dist_mat_xarr[0,:])
            
            for row, fiber in enumerate(fiber_labels):
                dash_min_idx = np.where(dist_matrix[row, :] == np.min(dist_matrix[row, :]))[0][0]
                dash_min = dash_labels[dash_min_idx]
                
                fiber_table_og.loc[(fiber_table_og.fov==fov) & (fiber_table_og.label==fiber), 'closet_duct_dash'] = dash_min
            
                fiber_orientation = duct_table[duct_table.label==fiber].orientation.values[0]
                dash_orientation = duct_segments[duct_segments.label==dash_min].orientation.values[0]
                duct_aligment = (np.sqrt((fiber_orientation - dash_orientation) ** 2))
                
                fiber_table_og.loc[(fiber_table_og.fov==fov) & (fiber_table_og.label==fiber), 'duct_alignment'] = duct_aligment
                
        progress.update(1)
        
fiber_table_all = fiber_table_all.merge(fiber_table_og[['fov', 'label', 'closet_duct_dash', 'duct_alignment']], on=['fov', 'label'], how='left')
fiber_table_all.to_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table-duct_alignment-length_ratio.csv', index=False)

In [175]:
fiber_table_all

,fov,label,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,area,eccentricity,euler_number,alignment_score,min_edge_dist,min_duct_index,min_duct_num,closet_duct_dash,duct_alignment
0,TA501_R3C3,1,49.529915,929.064103,22.736570,18.281580,-0.099037,234,0.594547,1,NaN,38.938735,0.0,1.0,NaN,NaN
1,TA501_R3C3,3,94.067308,1013.961538,20.365788,13.629157,1.460215,208,0.743066,1,NaN,52.149436,0.0,1.0,NaN,NaN
2,TA501_R3C3,5,131.117978,935.969101,43.799707,12.172885,-0.632625,356,0.960604,1,0.376656,1.356730,0.0,1.0,4.0,0.019852
3,TA501_R3C3,7,201.828889,934.317778,100.954214,31.083154,0.525554,900,0.951421,1,0.591944,46.733063,0.0,1.0,4.0,1.138327
4,TA501_R3C3,9,202.093023,890.751163,38.526961,16.048036,-0.012317,430,0.909117,1,0.386381,8.345497,0.0,1.0,4.0,0.600456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35129,TA587_R12C5,205,951.539216,241.735294,21.121515,13.202444,-0.584552,204,0.780568,1,NaN,645.857211,0.0,1.0,NaN,NaN
35130,TA587_R12C5,206,960.262840,337.090634,47.579264,10.135833,0.851364,331,0.977046,1,0.580400,656.662792,0.0,1.0,7.0,2.296914
35131,TA587_R12C5,208,972.925926,182.008230,52.446102,28.950942,-0.058408,972,0.833835,1,NaN,671.705943,0.0,1.0,NaN,NaN
35132,TA587_R12C5,213,981.464789,666.250704,26.942926,18.889293,0.976755,355,0.713077,1,NaN,578.759703,1.0,2.0,NaN,NaN


In [ ]:
fiber_table_all.to_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table-duct_alignment-length_ratio.csv', index=False)

In [ ]:
from ark.segmentation.marker_quantification import create_marker_count_matrices
from alpineer import load_utils, io_utils
import xarray as xr

channel = 'FAP'
# channel = 'COL1A1'

tiff_dir = '/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/image_data'
segmentation_dir = f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_all'

extraction='total_intensity'
mask_types = ['fiber_labels']
fast_extraction = True

fovs = np.unique(fiber_table_all.fov)

# define number of FOVs for batch processing
cohort_len = len(fovs)

# create the final dfs to store the processed data
normalized_tables = []

for fov_index, fov_name in enumerate(fovs):
    image_data = load_utils.load_imgs_from_tree(data_dir=tiff_dir,
                                                    img_sub_folder='',
                                                    fovs=[fov_name], channels=[channel])

    for mask_type in mask_types:
        # load the segmentation labels in
        fov_mask_name = fov_name + '_' + mask_type + ".tiff"
        current_labels_cell = load_utils.load_imgs_from_dir(data_dir=segmentation_dir,
                                                            files=[fov_mask_name],
                                                            xr_dim_name='compartments',
                                                            xr_channel_names=[mask_type],
                                                            trim_suffix='_' + mask_type)

        compartments = ['whole_cell']
        segmentation_labels = current_labels_cell.values

        current_labels = xr.DataArray(segmentation_labels,
                                      coords=[current_labels_cell.fovs,
                                              current_labels_cell.rows,
                                              current_labels_cell.cols,
                                              compartments],
                                      dims=current_labels_cell.dims)

        # segment the imaging data
        cell_table_size_normalized, cell_table_arcsinh_transformed = create_marker_count_matrices(
            segmentation_labels=current_labels,
            image_data=image_data,
            extraction=extraction,
            nuclear_counts=False,
            fast_extraction=fast_extraction,
        )

        # add mask type column to the data frame
        if mask_type == "final_cells_remaining":
            mask_type_str = "whole_cell"
        else:
            mask_type_str = mask_type
        cell_table_size_normalized['mask_type'] = mask_type_str

        # add to larger dataframe
        normalized_tables.append(cell_table_size_normalized)

# now append to the final dfs to return
combined_cell_table_size_normalized = pd.concat(normalized_tables)

combined_cell_table_size_normalized.to_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fibers_mean_intensities.csv', index=False)

In [180]:
channel = 'FAP'
# channel = 'COL1A1'
combined_cell_table_size_normalized = pd.read_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fibers_mean_intensities.csv')
fiber_table_all = pd.read_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table-duct_alignment-length_ratio.csv')

combined_cell_table_size_normalized = combined_cell_table_size_normalized.rename(columns={'FAP': 'FAP_mean_intensity'})
combined_cell_table_size_normalized = combined_cell_table_size_normalized[['fov', 'label', 'FAP_mean_intensity', 'cell_size']]

fiber_table_final = fiber_table_all.merge(combined_cell_table_size_normalized, on=['fov', 'label'])
fiber_table_final

,Unnamed: 0,fov,label,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,area,eccentricity,euler_number,alignment_score,min_edge_dist,min_duct_index,min_duct_num,closet_duct_dash,duct_alignment,FAP_mean_intensity,cell_size
0,0,TA501_R3C3,1,49.529915,929.064103,22.736570,18.281580,-0.099037,234,0.594547,1,NaN,38.938735,0.0,1.0,NaN,NaN,0.003183,234.0
1,1,TA501_R3C3,3,94.067308,1013.961538,20.365788,13.629157,1.460215,208,0.743066,1,NaN,52.149436,0.0,1.0,NaN,NaN,0.002429,208.0
2,2,TA501_R3C3,5,131.117978,935.969101,43.799707,12.172885,-0.632625,356,0.960604,1,0.376656,1.356730,0.0,1.0,4.0,0.019852,0.002729,356.0
3,3,TA501_R3C3,7,201.828889,934.317778,100.954214,31.083154,0.525554,900,0.951421,1,0.591944,46.733063,0.0,1.0,4.0,1.138327,0.004239,900.0
4,4,TA501_R3C3,9,202.093023,890.751163,38.526961,16.048036,-0.012317,430,0.909117,1,0.386381,8.345497,0.0,1.0,4.0,0.600456,0.004480,430.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35129,35129,TA587_R12C5,205,951.539216,241.735294,21.121515,13.202444,-0.584552,204,0.780568,1,NaN,645.857211,0.0,1.0,NaN,NaN,0.001764,204.0
35130,35130,TA587_R12C5,206,960.262840,337.090634,47.579264,10.135833,0.851364,331,0.977046,1,0.580400,656.662792,0.0,1.0,7.0,2.296914,0.000772,331.0
35131,35131,TA587_R12C5,208,972.925926,182.008230,52.446102,28.950942,-0.058408,972,0.833835,1,NaN,671.705943,0.0,1.0,NaN,NaN,0.002863,972.0
35132,35132,TA587_R12C5,213,981.464789,666.250704,26.942926,18.889293,0.976755,355,0.713077,1,NaN,578.759703,1.0,2.0,NaN,NaN,0.001212,355.0


In [185]:
channel = 'FAP'
# channel = 'COL1A1'
fiber_table_all = pd.read_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table-duct_alignment-length_ratio.csv')
fiber_table_all

,fov,label,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,area,eccentricity,euler_number,alignment_score,min_edge_dist,min_duct_index,min_duct_num,closet_duct_dash,duct_alignment
0,TA501_R3C3,1,49.529915,929.064103,22.736570,18.281580,-0.099037,234,0.594547,1,NaN,38.938735,0.0,1.0,NaN,NaN
1,TA501_R3C3,3,94.067308,1013.961538,20.365788,13.629157,1.460215,208,0.743066,1,NaN,52.149436,0.0,1.0,NaN,NaN
2,TA501_R3C3,5,131.117978,935.969101,43.799707,12.172885,-0.632625,356,0.960604,1,0.376656,1.356730,0.0,1.0,4.0,0.019852
3,TA501_R3C3,7,201.828889,934.317778,100.954214,31.083154,0.525554,900,0.951421,1,0.591944,46.733063,0.0,1.0,4.0,1.138327
4,TA501_R3C3,9,202.093023,890.751163,38.526961,16.048036,-0.012317,430,0.909117,1,0.386381,8.345497,0.0,1.0,4.0,0.600456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35129,TA587_R12C5,205,951.539216,241.735294,21.121515,13.202444,-0.584552,204,0.780568,1,NaN,645.857211,0.0,1.0,NaN,NaN
35130,TA587_R12C5,206,960.262840,337.090634,47.579264,10.135833,0.851364,331,0.977046,1,0.580400,656.662792,0.0,1.0,7.0,2.296914
35131,TA587_R12C5,208,972.925926,182.008230,52.446102,28.950942,-0.058408,972,0.833835,1,NaN,671.705943,0.0,1.0,NaN,NaN
35132,TA587_R12C5,213,981.464789,666.250704,26.942926,18.889293,0.976755,355,0.713077,1,NaN,578.759703,1.0,2.0,NaN,NaN


In [183]:
# channel = 'FAP'
channel = 'COL1A1'
fiber_table_all = pd.read_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table-duct_alignment-length_ratio.csv')
fiber_table_all = fiber_table_all.drop(columns='Unnamed: 0')
fiber_table_all.to_csv(f'/Volumes/Shared/Inna Averbukh/DCIS/Cohorts/20230317_DCIS/fiber_segmentation/{channel}/{channel}_fiber_object_table-duct_alignment-length_ratio.csv', index=False)
fiber_table_all

,fov,label,centroid-0,centroid-1,major_axis_length,minor_axis_length,orientation,area,eccentricity,euler_number,alignment_score,min_edge_dist,min_duct_index,min_duct_num,closet_duct_dash,duct_alignment
0,TA501_R3C3,1,49.529915,929.064103,22.736570,18.281580,-0.099037,234,0.594547,1,NaN,38.938735,0.0,1.0,NaN,NaN
1,TA501_R3C3,3,94.067308,1013.961538,20.365788,13.629157,1.460215,208,0.743066,1,NaN,52.149436,0.0,1.0,NaN,NaN
2,TA501_R3C3,5,131.117978,935.969101,43.799707,12.172885,-0.632625,356,0.960604,1,0.376656,1.356730,0.0,1.0,4.0,0.019852
3,TA501_R3C3,7,201.828889,934.317778,100.954214,31.083154,0.525554,900,0.951421,1,0.591944,46.733063,0.0,1.0,4.0,1.138327
4,TA501_R3C3,9,202.093023,890.751163,38.526961,16.048036,-0.012317,430,0.909117,1,0.386381,8.345497,0.0,1.0,4.0,0.600456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35129,TA587_R12C5,205,951.539216,241.735294,21.121515,13.202444,-0.584552,204,0.780568,1,NaN,645.857211,0.0,1.0,NaN,NaN
35130,TA587_R12C5,206,960.262840,337.090634,47.579264,10.135833,0.851364,331,0.977046,1,0.580400,656.662792,0.0,1.0,7.0,2.296914
35131,TA587_R12C5,208,972.925926,182.008230,52.446102,28.950942,-0.058408,972,0.833835,1,NaN,671.705943,0.0,1.0,NaN,NaN
35132,TA587_R12C5,213,981.464789,666.250704,26.942926,18.889293,0.976755,355,0.713077,1,NaN,578.759703,1.0,2.0,NaN,NaN


In [178]:
combined_cell_table_size_normalized

,cell_size,FAP,label,centroid-0,centroid-1,fov,mask_type
0,205.0,0.001022,4,12.682927,784.507317,TA501_R12C7,fiber_labels
1,515.0,0.000870,7,35.460194,1921.409709,TA501_R12C7,fiber_labels
2,1049.0,0.001138,13,64.060057,1507.382269,TA501_R12C7,fiber_labels
3,540.0,0.001259,27,78.474074,1719.150000,TA501_R12C7,fiber_labels
4,687.0,0.000923,29,79.590975,196.786026,TA501_R12C7,fiber_labels
...,...,...,...,...,...,...,...
35129,765.0,0.006298,134,922.439216,153.781699,TA587_R9C5,fiber_labels
35130,279.0,0.002910,140,951.749104,229.577061,TA587_R9C5,fiber_labels
35131,415.0,0.003826,142,961.922892,410.667470,TA587_R9C5,fiber_labels
35132,387.0,0.002353,143,979.666667,329.894057,TA587_R9C5,fiber_labels


In [ ]:
from skimage.measure import label
from alpineer import io_utils
from skimage.filters import gaussian
from skimage.morphology import binary_erosion, disk, dilation, binary_dilation

from natsort import natsorted
import pandas as pd
from tqdm.notebook import tqdm
import os
import natsort as ns
import skimage.io as io
from copy import deepcopy
import numpy as np
from skimage.measure import regionprops_table

import warnings
warnings.filterwarnings("ignore")

mask_dir = '/Users/csowers/.CMVolumes/google_drive/Shared with me/DCIS 2.0 Masking/DCIS_Normal_masks+myoep'
fov_list = io_utils.list_folders(mask_dir, substrs='TA')
fov_list = ns.natsorted(fov_list)

lots_of_ducts, duct_num = [], []
edges_dists, center_dists, ratio_dists = [], [], []
dfs = []
tables = []
mappings = []

with tqdm(total=len(fov_list), desc="Edge Distance Calculation", unit="FOV") \
as progress:
    for fov_num, fov in enumerate(fov_list):
        if fov_num%25==0:
            print(fov_num)

        # read in mask
        mask_path = os.path.join(mask_dir, fov, 'final_masks', 'ducts_labeled.tiff')
        try:
            #print(fov)
            ducts_labeled = io.imread(mask_path)
        except:
            print(f"Failed: {fov}")
            progress.update(1)
            continue

        # save duct mapping
        ducts = np.unique(ducts_labeled[ducts_labeled>0])

        # get duct centroids and outer boundary
        for i,duct_num in enumerate(ducts):

            object_mask = deepcopy(ducts_labeled)
            object_mask[object_mask != duct_num] = 0
            props = regionprops_table(object_mask, properties=('centroid', 'orientation', 'axis_major_length', 'axis_minor_length'))
            
            props_table = pd.DataFrame(props)
            props_table['fov'] = fov
            props_table['DuctNumber'] = duct_num
            
            tables.append(props_table)
        progress.update(1)

duct_table = pd.concat(tables)

In [27]:
duct_table = duct_table[['fov', 'DuctNumber', 'axis_major_length', 'axis_minor_length']]
duct_table = duct_table.rename(columns={'axis_major_length': 'duct_axis_major_length', 'axis_minor_length': 'duct_axis_minor_length'})

In [29]:
duct_table['duct_aspect_ratio'] = duct_table['duct_axis_major_length'] / duct_table['duct_axis_minor_length']

In [34]:
duct_table.to_csv('/Users/csowers/Documents/DCIS_dist_analysis/duct_aspect_ratio.csv', index=False)

In [35]:
duct_table

,fov,DuctNumber,duct_axis_major_length,duct_axis_minor_length,duct_aspect_ratio
0,TA501_R3C3,1,1296.066769,417.982851,3.100765
0,TA501_R4C7,1,670.820993,452.169887,1.483560
0,TA501_R4C7,3,503.717800,239.030663,2.107335
0,TA501_R4C7,6,339.272321,236.837418,1.432511
0,TA501_R6C1,1,2311.911358,1589.164233,1.454797
...,...,...,...,...,...
0,TA587_R12C1,20,758.439730,506.425099,1.497635
0,TA587_R12C2,2,762.347628,332.856869,2.290317
0,TA587_R12C4,4,753.419503,394.620754,1.909224
0,TA587_R12C5,1,413.032736,313.813332,1.316173
